In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
plt.rcParams['figure.dpi'] = 120 # Adjusting figure size

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation

In [ ]:
bit_file = 'bitcoin_updated.csv'
ether_file = 'ethereum_updated.csv'

In [ ]:
def process(file):
    df = pd.read_csv(file, header=0)
    df['Time'] = pd.to_datetime(df['Date'])
    df.drop(columns=['Date'], inplace=True)
    columns = ['Open*','High','Low','Close**','Volume', 'Market Cap']
    df = df[['Time','Open*','High','Low','Volume', 'Market Cap','Close**']]
    
    for col in columns:
        df[col] = df[col].str.replace(',', '').astype(float)
        #df[col] = df[col].replace(astype(float))
    df = df.sort_values(by='Time').set_index('Time')
    return df

In [ ]:
df = process(bit_file)
df = df[['Close**']]
train_df = df[:-10].copy()
test_df = df[-10:].copy()
train, test = train_df.values, test_df.values
    
train = np.reshape(train, (len(train), 1))
scale = MinMaxScaler()
train = scale.fit_transform(train)
x_train, y_train = train[0:len(train)-1], train[1:len(train)]
x_train = np.reshape(x_train, (len(x_train), 1, 1))
#df.plot(figsize=(15,5), title="Closing Price")    

In [ ]:
model = Sequential()
model.add(LSTM(128,activation="sigmoid",input_shape=(1,1)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=100, batch_size=50)

In [ ]:
model.summary()

In [ ]:
x_test = np.reshape(test, (len(test),1))
x_test = scale.transform(x_test)
x_test = np.reshape(x_test, (len(x_test),1,1))
y_pred = model.predict(x_test)
y_pred = scale.inverse_transform(y_pred)

In [ ]:
test_df['Close_pred'] = y_pred
data = pd.concat([test_df, train_df], sort=False)

In [ ]:
data.plot()

In [ ]:
test_df.plot()